In [1]:
from ultralytics import YOLO
import cv2
from sort.sort import *
import easyocr
from PIL import Image

In [3]:
coco_model = YOLO('./models/yolov8n.onnx')
mot_tracker = Sort()
license_plate_detector = YOLO('./models/license_plate_detector.onnx')

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.


In [4]:
reader = easyocr.Reader(['en'], gpu=False)

vehicles = [2, 3, 5, 6, 7]
plate_text_font = cv2.FONT_HERSHEY_SIMPLEX
plate_text_fontScale = 0.9
plate_text_color = (0, 255, 0)
plate_text_thickness = 2

Using CPU. Note: This module is much faster with a GPU.


In [40]:
def read_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)
    for detection in detections:
        bbox, text, score = detection
        text = text.upper().replace(' ', '')
        print("text detected : "+text)
        return text, score
    return "No_Plate",-1

In [41]:
def detect_license_plate(vehicle,image,vehicle_detected):
    # vehicle_x1, vehicle_y1, vehicle_x2, vehicle_y2, vehicle_score = vehicle_detected
    vehicle_x1, vehicle_y1, vehicle_x2, vehicle_y2, vehicle_id, vehicle_score, class_id = vehicle_detected
    numberplate_values = []
    numberplates = license_plate_detector(vehicle)[0]
    for numberplate in numberplates.boxes.data.tolist():
        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, plate_id = numberplate
        plate = vehicle[int(plate_y1):int(plate_y2) , int(plate_x1):int(plate_x2)]
        plate_height = plate_y2 - plate_y1
        plate_width = plate_x2 - plate_x1
        cv2.rectangle(image, (int(vehicle_x1 + plate_x1), int(vehicle_y1 + plate_y1)), (int(vehicle_x1 + plate_x1 + plate_width), int(vehicle_y1 + plate_y1 + plate_height)), (0, 255, 0), 2)
        plate_text, plate_text_score = read_plate(plate)
        plate_text_org = (int(vehicle_x1), int(vehicle_y1) + 25) 
        numberplate_values.append([[plate_x1, plate_y1, plate_x2, plate_y2],[vehicle_x1, vehicle_y1, vehicle_x2, vehicle_y2],[int(vehicle_x1 + plate_x1), int(vehicle_y1 + plate_y1), int(vehicle_x1 + plate_x1 + plate_width), int(vehicle_y1 + plate_y1 + plate_height)], plate_score, vehicle_score, plate_text_score, plate_text])
        cv2.putText(image, plate_text, plate_text_org, plate_text_font, plate_text_fontScale, plate_text_color, plate_text_thickness)
        # cv2.imshow("YOLOv8 Tracking", image)
    return numberplate_values

In [43]:
# Open the video file
video_path = "test-data/sample.mp4"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = coco_model.track(frame, persist=True, classes=vehicles)
        # print("\n ------------------------------------- results : -------------------------------\n",results)
        # print("\n -------------------------- results[0].boxes.data.tolist() -------------------- \n",results[0].boxes.data.tolist())
        # Visualize the results on the frame
        for detection in results[0].boxes.data.tolist():
            vehicle_x1, vehicle_y1, vehicle_x2, vehicle_y2, vehicle_id, vehicle_score, class_id = detection
            vehicle = frame[int(vehicle_y1):int(vehicle_y2), int(vehicle_x1):int(vehicle_x2)]
            print(detect_license_plate(vehicle,frame,detection))
            cv2.rectangle(frame, (int(vehicle_x1), int(vehicle_y1)), (int(vehicle_x2), int(vehicle_y2)), (0, 255, 0), 2)
            plate_text_org = (int(vehicle_x1), int(vehicle_y1) - 10) 
            cv2.putText(frame, ("vehicle id : "+str(vehicle_id)), plate_text_org, plate_text_font, plate_text_fontScale, plate_text_color, plate_text_thickness)
        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        # break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

0: 384x640 10 cars, 1 bus, 1 truck, 251.7ms
Speed: 6.0ms preprocess, 251.7ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 608x640 (no detections), 251.9ms
Speed: 6.0ms preprocess, 251.9ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)
[]

0: 544x640 (no detections), 232.8ms
Speed: 5.0ms preprocess, 232.8ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)
[]

0: 608x640 1 0, 293.3ms
Speed: 7.8ms preprocess, 293.3ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)
text detected : WJMRU
[[[85.82447814941406, 136.2405242919922, 148.3189239501953, 155.0635986328125], [249.43191528320312, 459.6938781738281, 476.0197448730469, 666.3910522460938], [335, 595, 397, 614], 0.7241805791854858, 0.8731617331504822, 0.09524374339383433, 'WJMRU']]

0: 640x640 (no detections), 632.5ms
Speed: 13.9ms preprocess, 632.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
[]

0: 640x640 1 0, 500.5ms
Speed: 15.0ms preproc

# Image

In [7]:
def vehicle_detection_and_recognition(image):
    detections = coco_model(image)[0]
    vehicle_detections = []
    for detection in detections.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = detection
        if int(class_id) in vehicles:
            vehicle_detections.append([x1, y1, x2, y2, score])
    # for vehicle_detected in vehicle_detections:
    #     vehicle_x1, vehicle_y1, vehicle_x2, vehicle_y2, vehicle_score = vehicle_detected
    #     vehicle = image[int(vehicle_y1):int(vehicle_y2), int(vehicle_x1):int(vehicle_x2)]
    #     cv2.rectangle(image, (int(vehicle_x1), int(vehicle_y1)), (int(vehicle_x2), int(vehicle_y2)), (0, 255, 0), 2)
    #     print(detect_license_plate(vehicle,image,vehicle_detected))
        print(vehicle_detections)

In [8]:
image = cv2.imread('test-data/image2.jpg')
vehicle_detection_and_recognition(image)
cv2.imshow('Image After Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Loading models\yolov8n.onnx for ONNX Runtime inference...
requirements: Ultralytics requirement ['onnxruntime'] not found, attempting AutoUpdate...
     ---------------------------------------- 46.0/46.0 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 86.8/86.8 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 7.3/7.3 MB 2.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 14.4s, installed 1 package: ['onnxruntime']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 640x640 4 cars, 290.7ms
Speed: 27.0ms preprocess, 290.7ms inference, 41.0ms postprocess per image at shape (1, 3, 640, 640)
[[566.7183227539062, 12.633399963378906, 1325.7371826171875, 482.9703369140625, 0.8249082565307617]]
[[566.7183227539062, 12.633399963378906, 1325.7371826171875, 482.9703369140625, 0.8249082565307617], [324.4737243652344, 103.93415069580078, 495.2755432128906, 232.47781372070312, 0.8116979002952576]]
[[566.7183227539062, 12.6333